# Multi-class classification

look at description of movie, and predict the genre that it might belong to  

In [1]:
import pandas as pd
import numpy as np

# Kaggle data link
get data from the Genre Classification dataset, combine train and test, and save it in movie_data.csv (commented out the code after saving the csv file)

link to the original data from kaggle - 
https://www.kaggle.com/datasets/hijest/genre-classification-dataset-imdb/data

In [4]:
train=pd.read_csv('Genre Classification Dataset/train_data.txt',sep=':::',header=None,names=['id','title','genre','description'], engine='python')
test=pd.read_csv('Genre Classification Dataset/test_data_solution.txt',sep=':::',header=None,names=['id','title','genre','description'], engine='python')
print(train.head(2))
print(test.head(2))

   id                           title       genre  \
0   1   Oscar et la dame rose (2009)       drama    
1   2                   Cupid (1997)    thriller    

                                         description  
0   Listening in to a conversation between his do...  
1   A brother and sister with a past incestuous r...  
   id                       title       genre  \
0   1       Edgar's Lunch (1998)    thriller    
1   2   La guerra de papá (1977)      comedy    

                                         description  
0   L.R. Brane loves his life - his car, his apar...  
1   Spain, March 1964: Quico is a very naughty ch...  


In [5]:
# 1. Select the desired columns:
train_selected = train[['title', 'genre', 'description']]
test_selected = test[['title', 'genre', 'description']]

# 2. Vertically concatenate (join) the DataFrames:
combined_df = pd.concat([train_selected, test_selected], ignore_index=True)
combined_df.to_csv('movie_data.csv', index=False)
print(combined_df.head(2))

                            title       genre  \
0   Oscar et la dame rose (2009)       drama    
1                   Cupid (1997)    thriller    

                                         description  
0   Listening in to a conversation between his do...  
1   A brother and sister with a past incestuous r...  


# Data cleaning and exploration

In [3]:
df = pd.read_csv('movie_data.csv')
print(df.head())
print(df.shape)

                                title       genre  \
0       Oscar et la dame rose (2009)       drama    
1                       Cupid (1997)    thriller    
2   Young, Wild and Wonderful (1980)       adult    
3              The Secret Sin (1915)       drama    
4             The Unrecovered (2007)       drama    

                                         description  
0   Listening in to a conversation between his do...  
1   A brother and sister with a past incestuous r...  
2   As the bus empties the students for their fie...  
3   To help their unemployed father make ends mee...  
4   The film's title refers not only to the un-re...  
(108414, 3)


### Checking for missing values

In [ ]:
print(pd.DataFrame({'Missing Count': df.isnull().sum(), 'Missing Percentage': (df.isnull().sum() / len(df)) * 100})) #Prints a dataframe with missing counts and percentages.

### Checking for class imbalance  

- Dataset is imbalanced, with majority classes being genres - 'drama', 'documentary', 'comedy', 'short', etc.  
- We will choose to leave the dataset as is without any undersampling or oversampling, to reflect real-world scenarios.  
- As there is class imbalance, accuracy would be a misleading performance metric, and thus we will use F1 score as the evaluation metric.
- some models will naturally perform better than others, because of class imbalance.
- can add 'future work' as trying out different ways to mitigate this problem and the results from each way.
- show class specific performance and comment on that in the paper.

In [ ]:
genre_count = df['genre'].value_counts()

print(genre_count)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

genre_count = df['genre'].value_counts()

plt.figure(figsize=(10, 6))
plt.bar(genre_count.index, genre_count.values)
plt.xlabel('Genre')
plt.ylabel('Count')
plt.title('Genre Distribution')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()

### Distribution of description lengths.
- appears to be normally distributed with slight skewness.

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

df['description_length'] = df['description'].apply(len)

# Create a distribution plot (histogram)
plt.figure(figsize=(10, 6))
sns.histplot(df['description_length'], bins=50, kde=True)  # bins and kde can be adjusted
plt.title('Distribution of Description Lengths')
plt.xlabel('Description Length')
plt.ylabel('Frequency')
plt.show()

# Data preprocessing
- lowercasing
- removing punctuation
- removing stopwords
- lemmatization

In [19]:
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
import re

# Download necessary NLTK resources (if not already downloaded)
nltk.download('stopwords')

# Initialize stemmer and stop words
stemmer = SnowballStemmer('english')
stop_words = set(stopwords.words('english'))

def preprocess_text(text):
    """Preprocesses a text string."""
    # Lowercase
    text = text.lower()
    # Remove punctuation and special characters
    text = re.sub(r'[^\w\s]', '', text)
    # Tokenize
    words = text.split()
    # Remove stop words and stem
    words = [stemmer.stem(word) for word in words if word not in stop_words]
    # Join words back into a string
    return ' '.join(words)

# Apply preprocessing to the 'description' column
df['processed_description'] = df['description'].apply(preprocess_text)

# Example: Print the original and processed descriptions for the first few rows
print("Original vs. Processed Descriptions:")
for i in range(5):  # Print the first 5 rows as an example.
    print(f"Original: {df['description'][i]}")
    print(f"Processed: {df['processed_description'][i]}")
    print("-" * 40)

# Now, use the 'processed_description' column for feature extraction.

[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Original vs. Processed Descriptions:
Original:  Listening in to a conversation between his doctor and parents, 10-year-old Oscar learns what nobody has the courage to tell him. He only has a few weeks to live. Furious, he refuses to speak to anyone except straight-talking Rose, the lady in pink he meets on the hospital stairs. As Christmas approaches, Rose uses her fantastical experiences as a professional wrestler, her imagination, wit and charm to allow Oscar to live life and love to the full, in the company of his friends Pop Corn, Einstein, Bacon and childhood sweetheart Peggy Blue.
Processed: listen convers doctor parent 10yearold oscar learn nobodi courag tell week live furious refus speak anyon except straighttalk rose ladi pink meet hospit stair christma approach rose use fantast experi profession wrestler imagin wit charm allow oscar live life love full compani friend pop corn einstein bacon childhood sweetheart peggi blue
----------------------------------------
Original:  A 

# Feature Extraction 1: TF-IDF

In [20]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Initialize TF-IDF Vectorizer
tfidf_vectorizer = TfidfVectorizer(max_features=5000)  # You can adjust max_features

# Fit and transform the processed descriptions
tfidf_features = tfidf_vectorizer.fit_transform(df['processed_description'])

# Convert to a DataFrame (optional, but can be helpful for inspection)
tfidf_df = pd.DataFrame(tfidf_features.toarray(), columns=tfidf_vectorizer.get_feature_names_out())

# Print the shape of the TF-IDF features
print("TF-IDF Feature Shape:", tfidf_features.shape)

# Example: Print the first few rows of the TF-IDF DataFrame (if you converted it)
# print(tfidf_df.head())

# Now, tfidf_features (or tfidf_df) contains the TF-IDF representation of your text data.

TF-IDF Feature Shape: (108414, 5000)


## Train-test split

- 90:10 split
- shuffle before splitting
- stratified split to maintain class distribution in train and test sets

In [21]:
from sklearn.model_selection import train_test_split

# Assuming 'tfidf_features' is your TF-IDF feature matrix and 'df['genre']' is your target variable
X_train, X_test, y_train, y_test = train_test_split(
    tfidf_features,
    df['genre'],
    test_size=0.1,  # 10% for testing
    random_state=42,
    shuffle=True,  # Shuffle the data before splitting
    stratify=df['genre'] # Stratified split based on the 'genre' column
)

# Print the shapes of the train and test sets
print("X_train shape:", X_train.shape)
print("X_test shape:", X_test.shape)
print("y_train shape:", y_train.shape)
print("y_test shape:", y_test.shape)

#Verify the distribution of genres in the training and testing sets.
print('\nTraining genre distribution:')
print(y_train.value_counts(normalize=True))
print('\nTesting genre distribution:')
print(y_test.value_counts(normalize=True))

X_train shape: (97572, 5000)
X_test shape: (10842, 5000)
y_train shape: (97572,)
y_test shape: (10842,)

Training genre distribution:
genre
drama           0.251117
documentary     0.241596
comedy          0.137375
short           0.093572
horror          0.040657
thriller        0.029342
action          0.024249
western         0.019042
reality-tv      0.016296
family          0.014451
adventure       0.014297
music           0.013487
romance         0.012401
sci-fi          0.011930
adult           0.010884
crime           0.009316
animation       0.009183
sport           0.007963
talk-show       0.007215
fantasy         0.005944
mystery         0.005873
musical         0.005104
biography       0.004878
history         0.004479
game-show       0.003567
news            0.003341
war             0.002439
Name: proportion, dtype: float64

Testing genre distribution:
genre
drama           0.251153
documentary     0.241561
comedy          0.137336
short           0.093617
horror          0

## Model 1 - Multinomial Naive Bayes

In [7]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report

# Initialize and train the Naive Bayes model
nb_classifier = MultinomialNB()
nb_classifier.fit(X_train, y_train)

# Make predictions on the test set
y_pred_nb = nb_classifier.predict(X_test)

# Evaluate the model
acc_nb = accuracy_score(y_test, y_pred_nb) 
pre_nb = precision_score(y_test, y_pred_nb, average='weighted') 
rec_nb = recall_score(y_test, y_pred_nb, average='weighted') 
f1_nb = f1_score(y_test, y_pred_nb, average='weighted') #using weighted average to account for class imbalance
report_nb = classification_report(y_test, y_pred_nb)

# Print the results
print("Naive Bayes Results:")
print(f"Accuracy: {acc_nb}")
print(f"Precision (weighted): {pre_nb}")
print(f"Recall (weighted): {rec_nb}")
print(f"F1-score (weighted): {f1_nb}")
print("Classification Report:")
print(report_nb)

Naive Bayes Results:
Accuracy: 0.538553772366722
Precision (weighted): 0.5133043959958458
Recall (weighted): 0.538553772366722
F1-score (weighted): 0.469730252107493
Classification Report:
               precision    recall  f1-score   support

      action        0.67      0.19      0.30       263
       adult        0.55      0.10      0.17       118
   adventure        0.75      0.14      0.23       155
   animation        0.00      0.00      0.00       100
   biography        0.00      0.00      0.00        53
      comedy        0.54      0.47      0.50      1489
       crime        0.00      0.00      0.00       101
 documentary        0.58      0.88      0.70      2619
       drama        0.47      0.81      0.60      2723
      family        0.33      0.01      0.01       157
     fantasy        0.00      0.00      0.00        65
   game-show        1.00      0.33      0.50        39
     history        0.00      0.00      0.00        49
      horror        0.73      0.44      

/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/m

## Model 2 - Logistic Regression

In [8]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report

# Initialize and train the Logistic Regression model
lr_classifier = LogisticRegression(max_iter=1000, random_state=42)  # Increase max_iter if needed
lr_classifier.fit(X_train, y_train)

# Make predictions on the test set
y_pred_lr = lr_classifier.predict(X_test)

# Evaluate the model
acc_lr = accuracy_score(y_test, y_pred_lr)
pre_lr = precision_score(y_test, y_pred_lr, average='weighted')
rec_lr = f1_score(y_test, y_pred_lr, average='weighted')
f1_lr = recall_score(y_test, y_pred_lr, average='weighted')
report_lr = classification_report(y_test, y_pred_lr)

# Print the results
print("Logistic Regression Results:")
print(f"Accuracy: {acc_lr}")
print(f"Precision (weighted): {pre_lr}")
print(f"Recall (weighted): {rec_lr}")
print(f"F1-score (weighted): {f1_lr}")
print("Classification Report:")
print(report_lr)

/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Logistic Regression Results:
Accuracy: 0.6074524995388305
Precision (weighted): 0.5896436123911377
Recall (weighted): 0.5767879175370361
F1-score (weighted): 0.6074524995388305
Classification Report:
               precision    recall  f1-score   support

      action        0.52      0.39      0.44       263
       adult        0.62      0.32      0.42       118
   adventure        0.60      0.24      0.34       155
   animation        0.51      0.19      0.28       100
   biography        0.00      0.00      0.00        53
      comedy        0.57      0.63      0.60      1489
       crime        0.32      0.06      0.10       101
 documentary        0.70      0.85      0.76      2619
       drama        0.57      0.76      0.65      2723
      family        0.53      0.16      0.25       157
     fantasy        0.33      0.08      0.12        65
   game-show        0.96      0.59      0.73        39
     history        1.00      0.02      0.04        49
      horror        0.67     

/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


## Model 3 - Support Vector Machine (SVM)

In [9]:
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report

# Initialize and train the Linear SVM model
linear_svm_classifier = LinearSVC(random_state=42, max_iter=1000)  # max_iter as needed
linear_svm_classifier.fit(X_train, y_train)

# Make predictions on the test set
y_pred_linear_svm = linear_svm_classifier.predict(X_test)

# Evaluate the model
acc_lsvm = accuracy_score(y_test, y_pred_linear_svm)
pre_lsvm = precision_score(y_test, y_pred_linear_svm, average='weighted')
rec_lsvm = recall_score(y_test, y_pred_linear_svm, average='weighted')
f1_lsvm = f1_score(y_test, y_pred_linear_svm, average='weighted')
report_linear_svm = classification_report(y_test, y_pred_linear_svm)

# Print the results
print("Linear SVM Results:")
print(f"Accuracy : {acc_lsvm}")
print(f"Precision (weighted): {pre_lsvm}")
print(f"Recall (weighted): {rec_lsvm}")
print(f"F1-score (weighted): {f1_lsvm}")
print("Classification Report:")
print(report_linear_svm)

Linear SVM Results:
Accuracy : 0.5945397528131341
Precision (weighted): 0.5653571873988986
Recall (weighted): 0.5945397528131341
F1-score (weighted): 0.5679010492541673
Classification Report:
               precision    recall  f1-score   support

      action        0.46      0.38      0.42       263
       adult        0.56      0.39      0.46       118
   adventure        0.47      0.25      0.33       155
   animation        0.35      0.20      0.25       100
   biography        0.00      0.00      0.00        53
      comedy        0.57      0.60      0.58      1489
       crime        0.29      0.10      0.15       101
 documentary        0.70      0.83      0.76      2619
       drama        0.58      0.73      0.64      2723
      family        0.42      0.17      0.24       157
     fantasy        0.29      0.11      0.16        65
   game-show        0.83      0.62      0.71        39
     history        0.67      0.04      0.08        49
      horror        0.62      0.67   

### hyperparameter tuning

In [ ]:
# from sklearn.svm import LinearSVC
# from sklearn.model_selection import GridSearchCV
# from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report

# # Define the parameter grid to search
# param_grid = {
#     'C': [0.01, 0.1, 1],  # Regularization parameter
#     'loss': ['hinge', 'squared_hinge'],  # Loss function
#     'dual': [True, False]  # Whether to use dual formulation
# }

# # Initialize the LinearSVC model
# linear_svm_classifier = LinearSVC(random_state=42, max_iter=1000)

# # Initialize GridSearchCV
# grid_search = GridSearchCV(estimator=linear_svm_classifier, param_grid=param_grid, 
#                            scoring='f1_weighted', cv=5, verbose=1, n_jobs=-1)

# # Perform grid search on the training data
# grid_search.fit(X_train, y_train)

# # Get the best parameters and model
# best_params = grid_search.best_params_
# best_model = grid_search.best_estimator_

# # Make predictions on the test set using the best model
# y_pred_linear_svm = best_model.predict(X_test)

# # Evaluate the model
# acc_lsvm_best = accuracy_score(y_test, y_pred_linear_svm)
# pre_lsvm_best = precision_score(y_test, y_pred_linear_svm, average='weighted')
# rec_lsvm_best = recall_score(y_test, y_pred_linear_svm, average='weighted')
# f1_lsvm_best = f1_score(y_test, y_pred_linear_svm, average='weighted')
# report_linear_svm = classification_report(y_test, y_pred_linear_svm)

# # Print the results
# print("Best Hyperparameters:", best_params)
# print("Linear SVM Results with Best Hyperparameters:")
# print(f"Accuracy : {acc_lsvm}")
# print(f"Precision (weighted): {pre_lsvm}")
# print(f"Recall (weighted): {rec_lsvm}")
# print(f"F1-score (weighted): {f1_lsvm}")
# print("Classification Report:")
# print(report_linear_svm)

## Model 4 - Random Forest

In [10]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report

# Initialize and train the Random Forest model
rf_classifier = RandomForestClassifier(random_state=42)
rf_classifier.fit(X_train, y_train)

# Make predictions on the test set
y_pred_rf = rf_classifier.predict(X_test)

# Evaluate the model
acc_rf = accuracy_score(y_test, y_pred_rf)
pre_rf = precision_score(y_test, y_pred_rf, average='weighted')
rec_rf = recall_score(y_test, y_pred_rf, average='weighted')
f1_rf = f1_score(y_test, y_pred_rf, average='weighted')
report_rf = classification_report(y_test, y_pred_rf)

# Print the results
print("Random Forest Results:")
print(f"accuracy: {acc_rf}")
print(f"Precision (weighted): {pre_rf}")
print(f"Recall (weighted): {rec_rf}")
print(f"F1-score (weighted): {f1_rf}")
print("Classification Report:")
print(report_rf)

Random Forest Results:
accuracy: 0.5092233905183545
Precision (weighted): 0.5474896805652754
Recall (weighted): 0.5092233905183545
F1-score (weighted): 0.43419036825798385
Classification Report:
               precision    recall  f1-score   support

      action        0.44      0.02      0.03       263
       adult        0.80      0.10      0.18       118
   adventure        0.71      0.17      0.28       155
   animation        0.00      0.00      0.00       100
   biography        0.00      0.00      0.00        53
      comedy        0.50      0.36      0.42      1489
       crime        1.00      0.02      0.04       101
 documentary        0.58      0.87      0.69      2619
       drama        0.43      0.82      0.57      2723
      family        0.80      0.03      0.05       157
     fantasy        0.00      0.00      0.00        65
   game-show        0.91      0.54      0.68        39
     history        0.00      0.00      0.00        49
      horror        0.66      0.24

/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/m

In [ ]:
### hyperparameter tuning

In [ ]:
# from sklearn.ensemble import RandomForestClassifier
# from sklearn.model_selection import GridSearchCV
# from sklearn.metrics import f1_score, classification_report

# # Define the parameter grid to search
# param_grid = {
#     'n_estimators': [50, 100, 200],  # Number of trees in the forest
#     'max_depth': [None, 10, 20, 30],  # Maximum depth of the tree
#     'min_samples_split': [2, 5, 10],  # Minimum number of samples required to split an internal node
#     'min_samples_leaf': [1, 2, 4],  # Minimum number of samples required to be at a leaf node
#     'max_features': ['auto', 'sqrt', 'log2'],  # Number of features to consider when looking for the best split
#     'bootstrap': [True, False]  # Whether bootstrap samples are used when building trees
# }

# # Initialize the RandomForestClassifier
# rf_classifier = RandomForestClassifier(random_state=42)

# # Initialize GridSearchCV
# grid_search = GridSearchCV(estimator=rf_classifier, param_grid=param_grid, 
#                            scoring='f1_weighted', cv=5, verbose=1, n_jobs=-1)

# # Perform grid search on the training data
# grid_search.fit(X_train, y_train)

# # Get the best parameters and model
# best_params = grid_search.best_params_
# best_model = grid_search.best_estimator_

# # Make predictions on the test set using the best model
# y_pred_rf = best_model.predict(X_test)

# # Evaluate the model
# f1_rf = f1_score(y_test, y_pred_rf, average='weighted')
# report_rf = classification_report(y_test, y_pred_rf)

# # Print the results
# print("Best Hyperparameters:", best_params)
# print("Random Forest Results with Best Hyperparameters:")
# print(f"F1-score (weighted): {f1_rf}")
# print("Classification Report:")
# print(report_rf)

## Model 5 - XGBoost

In [12]:
import xgboost as xgb
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report
from sklearn.preprocessing import LabelEncoder

# Label encode the target variable
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)
y_test_encoded = label_encoder.transform(y_test)

# Initialize and train the XGBoost model
xgb_classifier = xgb.XGBClassifier(random_state=42)
xgb_classifier.fit(X_train, y_train_encoded)

# Make predictions on the test set
y_pred_xgb_encoded = xgb_classifier.predict(X_test)

# Decode the predictions back to original labels
y_pred_xgb = label_encoder.inverse_transform(y_pred_xgb_encoded)

# Evaluate the model
accuracy_xgb = accuracy_score(y_test, y_pred_xgb)
precision_xgb = precision_score(y_test, y_pred_xgb, average='weighted')
recall_xgb = recall_score(y_test, y_pred_xgb, average='weighted')
f1_xgb = f1_score(y_test, y_pred_xgb, average='weighted')
report_xgb = classification_report(y_test, y_pred_xgb)

# Print the results
print("XGBoost Results:")
print(f"F1-score (weighted): {accuracy_xgb}")
print(f"F1-score (weighted): {precision_xgb}")
print(f"F1-score (weighted): {recall_xgb}")
print(f"F1-score (weighted): {f1_xgb}")
print("Classification Report:")
print(report_xgb)

XGBoost Results:
F1-score (weighted): 0.5667773473528869
F1-score (weighted): 0.5488471706977178
F1-score (weighted): 0.5667773473528869
F1-score (weighted): 0.5344488497740835
Classification Report:
               precision    recall  f1-score   support

      action        0.45      0.28      0.35       263
       adult        0.51      0.30      0.38       118
   adventure        0.54      0.26      0.35       155
   animation        0.40      0.14      0.21       100
   biography        1.00      0.02      0.04        53
      comedy        0.54      0.50      0.52      1489
       crime        0.20      0.05      0.08       101
 documentary        0.66      0.81      0.73      2619
       drama        0.51      0.77      0.61      2723
      family        0.47      0.13      0.20       157
     fantasy        0.25      0.06      0.10        65
   game-show        0.92      0.56      0.70        39
     history        0.00      0.00      0.00        49
      horror        0.65     

## Model 6 - Neural Network

In [7]:
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
import numpy as np
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report

# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(
    tfidf_features, df['genre'], test_size=0.1, random_state=42, stratify=df['genre']
)

# Label encode and one-hot encode the target variable
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)
y_test_encoded = label_encoder.transform(y_test)

onehot_encoder = OneHotEncoder(sparse_output=False)
y_train_onehot = onehot_encoder.fit_transform(y_train_encoded.reshape(-1, 1))
y_test_onehot = onehot_encoder.transform(y_test_encoded.reshape(-1, 1))

# Create the neural network model with specified parameters
model = keras.Sequential([
    keras.layers.Dense(128, activation='relu', input_shape=(X_train.shape[1],)),
    keras.layers.Dropout(0.3),
    keras.layers.Dense(64, activation='relu'), # half of 128
    keras.layers.Dropout(0.15), # half of 0.3
    keras.layers.Dense(y_train_onehot.shape[1], activation='softmax')
])

optimizer = keras.optimizers.Adam(learning_rate=0.001)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train.toarray(), y_train_onehot, epochs=5, batch_size=32, verbose=1)

# Evaluate the model on the test set
y_pred_test_onehot = model.predict(X_test.toarray())
y_pred_test = np.argmax(y_pred_test_onehot, axis=1)
y_test_decoded = np.argmax(y_test_onehot, axis=1)

acc_test = accuracy_score(y_test_decoded, y_pred_test)
pre_test = precision_score(y_test_decoded, y_pred_test, average='weighted')
rec_test = recall_score(y_test_decoded, y_pred_test, average='weighted')
f1_test = f1_score(y_test_decoded, y_pred_test, average='weighted')
report_test = classification_report(y_test_decoded, y_pred_test)

print("Neural Network Test Results:")
print(f"Accuracy: {acc_test}")
print(f"Precision (weighted): {pre_test}")
print(f"Recall(weighted): {rec_test}")
print(f"F1-score (weighted): {f1_test}")
print("Classification Report:\n", report_test)

/opt/conda/lib/python3.12/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


: 

In [ ]:
# scikit learn mlp

In [18]:
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import numpy as np
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report

# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(
    tfidf_features, df['genre'], test_size=0.1, random_state=42, stratify=df['genre']
)

# Convert TF-IDF sparse matrices to NumPy arrays
X_train_array = X_train.toarray()
X_test_array = X_test.toarray()

# Label encode the target variable
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)
y_test_encoded = label_encoder.transform(y_test)

# Create the MLP model with specified parameters
model = MLPClassifier(hidden_layer_sizes=(128, 64), activation='relu', solver='adam',
                      alpha=0.0001, batch_size=32, learning_rate='adaptive', max_iter=5,
                      random_state=42)

# Train the model
model.fit(X_train_array, y_train_encoded)

# Evaluate the model on the test set
y_pred_test = model.predict(X_test_array)

acc_test = accuracy_score(y_test_encoded, y_pred_test)
pre_test = precision_score(y_test_encoded, y_pred_test, average='weighted')
rec_test = recall_score(y_test_encoded, y_pred_test, average='weighted')
f1_test = f1_score(y_test_encoded, y_pred_test, average='weighted')
report_test = classification_report(y_test_encoded, y_pred_test)

print("scikit-learn MLP Test Results:")
print(f"Accuracy: {acc_test}")
print(f"Precision (weighted): {pre_test}")
print(f"Recall(weighted): {rec_test}")
print(f"F1-score (weighted): {f1_test}")
print("Classification Report:\n", report_test)

NameError: name 'tfidf_features' is not defined

# Word Embeddings (GLoVe6B - 100d)

In [7]:
import numpy as np
import pandas as pd  # Assuming you're using pandas DataFrames
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

# 1. Load GloVe Embeddings
def load_glove_embeddings(glove_file):
    embeddings = {}
    with open(glove_file, 'r', encoding='utf-8') as f:
        for line in f:
            values = line.split()
            word = values[0]
            vector = np.asarray(values[1:], dtype='float32')
            embeddings[word] = vector
    return embeddings

glove_embeddings = load_glove_embeddings('glove.6B.100d.txt')  # Replace with your path
embeddings_dim = 100  # Adjust to your GloVe dimension

# 2. Convert Text to Embedding Vectors
def text_to_embedding_vectors(texts, embeddings, embeddings_dim):
    embedding_vectors = []
    for text in texts:
        words = text.split()  # Assuming already lowercased and processed
        vectors = [embeddings.get(word, np.zeros(embeddings_dim)) for word in words]
        if vectors:
            text_vector = np.mean(vectors, axis=0)
        else:
            text_vector = np.zeros(embeddings_dim)
        embedding_vectors.append(text_vector)
    return np.array(embedding_vectors)

X_embedded = text_to_embedding_vectors(df['processed_description'], glove_embeddings, embeddings_dim)

# 3. Prepare Data and Train/Test Split
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(df['genre'])
onehot_encoder = OneHotEncoder(sparse_output=False)
y_onehot = onehot_encoder.fit_transform(y_encoded.reshape(-1, 1))

X_train_embedded, X_test_embedded, y_train_onehot, y_test_onehot = train_test_split(
    X_embedded, y_onehot, test_size=0.1, random_state=42, stratify=df['genre']
)

# X_train_embedded and X_test_embedded now contain the GloVe embedding vectors,
# y_train_onehot and y_test_onehot contain the one-hot encoded genre labels.
# You can use these variables to train and evaluate any machine learning model.

## Model 1 - Naive Bayes

In [8]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report
from sklearn.preprocessing import LabelEncoder
import numpy as np

# Assuming you have X_train_embedded, X_test_embedded, y_train_onehot, y_test_onehot from the previous GloVe embedding code

# Convert one-hot encoded labels back to label encoded format for Naive Bayes
y_train_encoded = np.argmax(y_train_onehot, axis=1)
y_test_encoded = np.argmax(y_test_onehot, axis=1)

# Initialize and train the Naive Bayes model
nb_classifier = MultinomialNB()

# Naive Bayes requires non-negative input, so we'll use absolute values
# GloVe embeddings can have negative values.
X_train_embedded_abs = np.abs(X_train_embedded)
X_test_embedded_abs = np.abs(X_test_embedded)

nb_classifier.fit(X_train_embedded_abs, y_train_encoded)

# Make predictions on the test set
y_pred_nb = nb_classifier.predict(X_test_embedded_abs)

# Evaluate the model
acc_nb = accuracy_score(y_test_encoded, y_pred_nb)
pre_nb = precision_score(y_test_encoded, y_pred_nb, average='weighted')
rec_nb = recall_score(y_test_encoded, y_pred_nb, average='weighted')
f1_nb = f1_score(y_test_encoded, y_pred_nb, average='weighted')
report_nb = classification_report(y_test_encoded, y_pred_nb)

# Print the results
print("Naive Bayes with GloVe Embeddings Results:")
print(f"Accuracy: {acc_nb}")
print(f"Precision (weighted): {pre_nb}")
print(f"Recall (weighted): {rec_nb}")
print(f"F1-score (weighted): {f1_nb}")
print("Classification Report:\n", report_nb)

Naive Bayes with GloVe Embeddings Results:
Accuracy: 0.3827707065117137
Precision (weighted): 0.1916778371891662
Recall (weighted): 0.3827707065117137
F1-score (weighted): 0.25423458140320393
Classification Report:
               precision    recall  f1-score   support

           0       0.00      0.00      0.00       263
           1       0.00      0.00      0.00       118
           2       0.00      0.00      0.00       155
           3       0.00      0.00      0.00       100
           4       0.00      0.00      0.00        53
           5       0.00      0.00      0.00      1489
           6       0.00      0.00      0.00       101
           7       0.43      0.75      0.55      2619
           8       0.35      0.80      0.48      2723
           9       0.00      0.00      0.00       157
          10       0.00      0.00      0.00        65
          11       0.00      0.00      0.00        39
          12       0.00      0.00      0.00        49
          13       0.00    

/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/m

# Logistic regression

In [10]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report
import numpy as np

# Assuming you have X_train_embedded, X_test_embedded, y_train_onehot, y_test_onehot from the GloVe embedding code

# Convert one-hot encoded labels back to label encoded format for Logistic Regression
y_train_encoded = np.argmax(y_train_onehot, axis=1)
y_test_encoded = np.argmax(y_test_onehot, axis=1)

# Initialize and train the Logistic Regression model
lr_classifier = LogisticRegression(random_state=42, max_iter=1000) # Increased max_iter

lr_classifier.fit(X_train_embedded, y_train_encoded)

# Make predictions on the test set
y_pred_lr = lr_classifier.predict(X_test_embedded)

# Evaluate the model
acc_lr = accuracy_score(y_test_encoded, y_pred_lr)
pre_lr = precision_score(y_test_encoded, y_pred_lr, average='weighted')
rec_lr = recall_score(y_test_encoded, y_pred_lr, average='weighted')
f1_lr = f1_score(y_test_encoded, y_pred_lr, average='weighted')
report_lr = classification_report(y_test_encoded, y_pred_lr)

# Print the results
print("Logistic Regression with GloVe Embeddings Results:")
print(f"Accuracy: {acc_lr}")
print(f"Precision (weighted): {pre_lr}")
print(f"Recall (weighted): {rec_lr}")
print(f"F1-score (weighted): {f1_lr}")
print("Classification Report:\n", report_lr)

Logistic Regression with GloVe Embeddings Results:
Accuracy: 0.4961261759822911
Precision (weighted): 0.4587705069747756
Recall (weighted): 0.4961261759822911
F1-score (weighted): 0.45639040932674213
Classification Report:
               precision    recall  f1-score   support

           0       0.43      0.29      0.35       263
           1       0.31      0.11      0.16       118
           2       0.15      0.04      0.06       155
           3       0.42      0.15      0.22       100
           4       0.00      0.00      0.00        53
           5       0.40      0.42      0.41      1489
           6       0.19      0.03      0.05       101
           7       0.58      0.77      0.66      2619
           8       0.49      0.69      0.57      2723
           9       0.38      0.06      0.11       157
          10       0.14      0.03      0.05        65
          11       0.72      0.59      0.65        39
          12       0.33      0.02      0.04        49
          13       

## Model 3 - Linear SVM

In [11]:
from sklearn.svm import LinearSVC
from sklearn.metrics import f1_score, classification_report, accuracy_score, precision_score, recall_score
import numpy as np

# Assuming you have X_train_embedded, X_test_embedded, y_train_onehot, y_test_onehot from the GloVe embedding code

# Convert one-hot encoded labels back to label encoded format for Linear SVM
y_train_encoded = np.argmax(y_train_onehot, axis=1)
y_test_encoded = np.argmax(y_test_onehot, axis=1)

# Initialize and train the Linear SVM model
svm_classifier = LinearSVC(random_state=42, max_iter=1000)  # Increased max_iter
svm_classifier.fit(X_train_embedded, y_train_encoded)

# Make predictions on the test set
y_pred_svm = svm_classifier.predict(X_test_embedded)

# Evaluate the model
acc_svm = accuracy_score(y_test_encoded, y_pred_svm)
pre_svm = precision_score(y_test_encoded, y_pred_svm, average='weighted')
rec_svm = recall_score(y_test_encoded, y_pred_svm, average='weighted')
f1_svm = f1_score(y_test_encoded, y_pred_svm, average='weighted')
report_svm = classification_report(y_test_encoded, y_pred_svm)

# Print the results
print("Linear SVM with GloVe Embeddings Results:")
print(f"Accuracy: {acc_svm:.4f}")
print(f"Precision (weighted): {pre_svm:.4f}")
print(f"Recall (weighted): {rec_svm:.4f}")
print(f"F1-score (weighted): {f1_svm:.4f}")
print("Classification Report:\n", report_svm)

Linear SVM with GloVe Embeddings Results:
Accuracy: 0.4803
Precision (weighted): 0.4164
Recall (weighted): 0.4803
F1-score (weighted): 0.4141
Classification Report:
               precision    recall  f1-score   support

           0       0.49      0.16      0.25       263
           1       0.57      0.03      0.06       118
           2       0.00      0.00      0.00       155
           3       0.00      0.00      0.00       100
           4       0.00      0.00      0.00        53
           5       0.39      0.36      0.37      1489
           6       0.00      0.00      0.00       101
           7       0.53      0.82      0.65      2619
           8       0.46      0.71      0.56      2723
           9       0.00      0.00      0.00       157
          10       0.00      0.00      0.00        65
          11       0.74      0.59      0.66        39
          12       0.00      0.00      0.00        49
          13       0.43      0.28      0.34       441
          14       0.42

/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/m

## Model 4 - Random Forest

In [17]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score, classification_report, accuracy_score, precision_score, recall_score
import numpy as np

# Assuming you have X_train_embedded, X_test_embedded, y_train_onehot, y_test_onehot from the GloVe embedding code

# Convert one-hot encoded labels back to label encoded format for Random Forest
y_train_encoded = np.argmax(y_train_onehot, axis=1)
y_test_encoded = np.argmax(y_test_onehot, axis=1)

# Initialize and train the Random Forest model
rf_classifier = RandomForestClassifier(random_state=42)

rf_classifier.fit(X_train_embedded, y_train_encoded)

# Make predictions on the test set
y_pred_rf = rf_classifier.predict(X_test_embedded)

# Evaluate the model
acc_rf = accuracy_score(y_test_encoded, y_pred_rf)
pre_rf = precision_score(y_test_encoded, y_pred_rf, average='weighted')
rec_rf = recall_score(y_test_encoded, y_pred_rf, average='weighted')
f1_rf = f1_score(y_test_encoded, y_pred_rf, average='weighted')
report_rf = classification_report(y_test_encoded, y_pred_rf)

# Print the results
print("Random Forest with GloVe Embeddings Results:")
print(f"Accuracy: {acc_rf}")
print(f"Precision (weighted): {pre_rf}")
print(f"Recall (weighted): {rec_rf}")
print(f"F1-score (weighted): {f1_rf}")
print("Classification Report:\n", report_rf)

Random Forest with GloVe Embeddings Results:
Accuracy: 0.4517616675890057
Precision (weighted): 0.4746915934366851
Recall (weighted): 0.4517616675890057
F1-score (weighted): 0.3754167299667026
Classification Report:
               precision    recall  f1-score   support

           0       0.33      0.02      0.04       263
           1       0.00      0.00      0.00       118
           2       1.00      0.02      0.04       155
           3       0.00      0.00      0.00       100
           4       0.00      0.00      0.00        53
           5       0.36      0.32      0.34      1489
           6       0.00      0.00      0.00       101
           7       0.50      0.83      0.62      2619
           8       0.42      0.70      0.53      2723
           9       0.67      0.01      0.03       157
          10       0.00      0.00      0.00        65
          11       0.94      0.44      0.60        39
          12       0.00      0.00      0.00        49
          13       0.49   

/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/m

## Model 5 - XGBoost

In [14]:
import xgboost as xgb
from sklearn.metrics import f1_score, classification_report, accuracy_score, precision_score, recall_score
import numpy as np

# Assuming you have X_train_embedded, X_test_embedded, y_train_onehot, y_test_onehot from the GloVe embedding code

# Convert one-hot encoded labels back to label encoded format for XGBoost
y_train_encoded = np.argmax(y_train_onehot, axis=1)
y_test_encoded = np.argmax(y_test_onehot, axis=1)

# Initialize and train the XGBoost model
xgb_classifier = xgb.XGBClassifier(random_state=42)

xgb_classifier.fit(X_train_embedded, y_train_encoded)

# Make predictions on the test set
y_pred_xgb = xgb_classifier.predict(X_test_embedded)

# Evaluate the model
acc_xgb = accuracy_score(y_test_encoded, y_pred_xgb)
pre_xgb = precision_score(y_test_encoded, y_pred_xgb, average='weighted')
rec_xgb = recall_score(y_test_encoded, y_pred_xgb, average='weighted')
f1_xgb = f1_score(y_test_encoded, y_pred_xgb, average='weighted')
report_xgb = classification_report(y_test_encoded, y_pred_xgb)

# Print the results
print("XGBoost with GloVe Embeddings Results:")
print(f"F1-score (weighted): {acc_xgb}")
print(f"F1-score (weighted): {pre_xgb}")
print(f"F1-score (weighted): {rec_xgb}")
print(f"F1-score (weighted): {f1_xgb}")
print("Classification Report:\n", report_xgb)

XGBoost with GloVe Embeddings Results:
F1-score (weighted): 0.48930086699870873
F1-score (weighted): 0.46097652509648374
F1-score (weighted): 0.48930086699870873
F1-score (weighted): 0.4526596865964386
Classification Report:
               precision    recall  f1-score   support

           0       0.40      0.29      0.33       263
           1       0.28      0.08      0.13       118
           2       0.37      0.12      0.18       155
           3       0.54      0.13      0.21       100
           4       0.00      0.00      0.00        53
           5       0.40      0.44      0.41      1489
           6       0.21      0.03      0.05       101
           7       0.57      0.77      0.66      2619
           8       0.48      0.65      0.55      2723
           9       0.33      0.06      0.11       157
          10       0.11      0.02      0.03        65
          11       0.84      0.54      0.66        39
          12       0.00      0.00      0.00        49
          13     

## Model 6 - Neural Network

In [ ]:
import tensorflow as tf
from tensorflow import keras
from sklearn.metrics import f1_score, classification_report, accuracy_score, precision_score, recall_score
import numpy as np

# Assuming you have X_train_embedded, X_test_embedded, y_train_onehot, y_test_onehot from the GloVe embedding code

# Create the neural network model with what I think are good parameters
model = keras.Sequential([
    keras.layers.Dense(256, activation='relu', input_shape=(X_train_embedded.shape[1],)),  # Increased hidden units
    keras.layers.Dropout(0.4),  # Increased dropout
    keras.layers.Dense(128, activation='relu'),
    keras.layers.Dropout(0.3), # Increased dropout
    keras.layers.Dense(y_train_onehot.shape[1], activation='softmax')
])

optimizer = keras.optimizers.Adam(learning_rate=0.001)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train_embedded, y_train_onehot, epochs=10, batch_size=32, verbose=1) # Increased epochs

# Evaluate the model on the test set
y_pred_test_onehot = model.predict(X_test_embedded)
y_pred_test = np.argmax(y_pred_test_onehot, axis=1)
y_test_decoded = np.argmax(y_test_onehot, axis=1)

f1_test = f1_score(y_test_decoded, y_pred_test, average='weighted')
report_test = classification_report(y_test_decoded, y_pred_test)

print("Neural Network with GloVe Embeddings Results:")
print(f"F1-score (weighted): {f1_test}")
print("Classification Report:\n", report_test)

In [ ]:
# scikit learn mlp

In [16]:
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import f1_score, classification_report, accuracy_score, precision_score, recall_score
import numpy as np

# Assuming you have X_train_embedded, X_test_embedded, y_train_onehot, y_test_onehot from the GloVe embedding code

# Convert one-hot encoded labels to integer labels
y_train = np.argmax(y_train_onehot, axis=1)
y_test = np.argmax(y_test_onehot, axis=1)

# Create the MLP model with improved parameters
model = MLPClassifier(hidden_layer_sizes=(512, 256, 128), activation='relu', solver='adam',
                      alpha=0.0001, batch_size=64, learning_rate='adaptive', max_iter=200,
                      random_state=42)

# Train the model
model.fit(X_train_embedded, y_train)

# Evaluate the model on the test set
y_pred_test = model.predict(X_test_embedded)

# Calculate evaluation metrics
accuracy_test = accuracy_score(y_test, y_pred_test)
precision_test = precision_score(y_test, y_pred_test, average='weighted')
recall_test = recall_score(y_test, y_pred_test, average='weighted')
f1_test = f1_score(y_test, y_pred_test, average='weighted')
report_test = classification_report(y_test, y_pred_test)

print("scikit-learn MLP with GloVe Embeddings Results:")
print(f"Accuracy: {accuracy_test}")
print(f"Precision (weighted): {precision_test}")
print(f"Recall (weighted): {recall_test}")
print(f"F1-score (weighted): {f1_test}")
print("Classification Report:\n", report_test)

scikit-learn MLP with GloVe Embeddings Results:
Accuracy: 0.421877882309537
Precision (weighted): 0.41426699998270117
Recall (weighted): 0.421877882309537
F1-score (weighted): 0.4174792924874099
Classification Report:
               precision    recall  f1-score   support

           0       0.33      0.34      0.34       263
           1       0.21      0.18      0.19       118
           2       0.21      0.24      0.22       155
           3       0.21      0.22      0.21       100
           4       0.00      0.00      0.00        53
           5       0.35      0.37      0.36      1489
           6       0.16      0.10      0.12       101
           7       0.59      0.63      0.61      2619
           8       0.48      0.48      0.48      2723
           9       0.15      0.13      0.14       157
          10       0.05      0.06      0.05        65
          11       0.58      0.56      0.57        39
          12       0.03      0.02      0.02        49
          13       0.36 

/opt/conda/lib/python3.12/site-packages/sklearn/neural_network/_multilayer_perceptron.py:697: UserWarning: Training interrupted by user.
  warnings.warn("Training interrupted by user.")


# Transformer (DistilBERT)`
- ran in google colab - weighted F-score: 0.61

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from transformers import DistilBertTokenizer, TFDistilBertForSequenceClassification
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import f1_score, classification_report, accuracy_score, precision_score, recall_score

# 1. Data Preparation and Preprocessing
label_encoder = LabelEncoder()
df['genre_encoded'] = label_encoder.fit_transform(df['genre'])

X_train, X_test, y_train, y_test = train_test_split(
    df['processed_description'], df['genre_encoded'], test_size=0.1, random_state=42, stratify=df['genre']
)

# 2. Tokenization
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')

def tokenize_data(texts, max_length=128):
    input_ids = []
    attention_masks = []
    for text in texts:
        encoded = tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=max_length,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            return_tensors='tf'
        )
        input_ids.append(encoded['input_ids'][0])
        attention_masks.append(encoded['attention_mask'][0])
    return tf.stack(input_ids), tf.stack(attention_masks)

X_train_ids, X_train_masks = tokenize_data(X_train)
X_test_ids, X_test_masks = tokenize_data(X_test)

# 3. Transformer Model
model = TFDistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=len(df['genre'].unique()))

# 4. Training
optimizer = tf.keras.optimizers.Adam(learning_rate=2e-5)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
model.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])

model.fit(
    [X_train_ids, X_train_masks],
    y_train,
    epochs=3,
    batch_size=16,
    validation_split=0.1
)

# 5. Evaluation
logits = model.predict([X_test_ids, X_test_masks]).logits
y_pred = np.argmax(logits, axis=1)

acc = f1_score(y_test, y_pred)
pre = f1_score(y_test, y_pred, average='weighted')
rec = f1_score(y_test, y_pred, average='weighted')
f1 = f1_score(y_test, y_pred, average='weighted')
report = classification_report(y_test, y_pred, target_names=label_encoder.classes_)

print(f"F1-score (weighted): {acc}")
print(f"F1-score (weighted): {pre}")
print(f"F1-score (weighted): {rec}")
print(f"F1-score (weighted): {f1}")
print(f"Classification Report:\n{report}")